# Imports

In [1]:
import numpy as np

In [2]:
from run_6body_encounter import run_6_body_simulation
from gif_plotting import visualize_frames

/Users/eirinichrysovergi/miniforge3/lib/python3.12/site-packages/amuse/support/options.py:11: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


# Examples of 6 body evolution (3 binaries)

## No encounter

In [3]:
# Initial conditions
age = 3.5 #Myr
masses = [90, 10,  20, 70,  10, 10] #Msun
sep= [30, 20, 50] #AU
ecc = [0.0, 0.0, 0]
centers = [[-100, 0, 0], [300, 0, 0], [0, 600, 0]] #AU
v_coms = [[30., 3., 0.], [-10., -21., 0.], [5., -20., 1.] ] # km/s
direction = [0.4, -0.6, 1.2]


no_colission , _, _= run_6_body_simulation(age, masses,sep,ecc, direction, centers, v_coms, "Test_No_Collision")

Starting simulation
Most massive star: Mass = 65.88 MSun, Velocity = 33.75 km/s


In [4]:
visualize_frames(no_colission, "Test_No collision")

 GIF saved as Gif-6body/encounter_evolution_Test_No collision.gif


## Encounter

In [5]:
# Initial conditions
# fast-test settings (use low SPH resolution for debugging)
age = 3.5  # Myr

# Six stars (Msun). Total ~220 Msun so a >100 Msun remnant is possible after 2+ mergers.
masses = [80, 40,   30, 20,    40, 10]   # [A1,A2, B1,B2, C1,C2] in MSun

# Tight inner binary (A) to encourage a prompt merger; other binaries moderately tight.
sep = [0.5, 1.0, 1.5]   # AU (A is very hard: 0.5 AU)

# Moderate-high eccentricity for the tight binary to encourage pericenter collisions.
ecc = [0.6, 0.2, 0.1]

# Place the three binaries relatively close so cross-binary interactions happen quickly.
centers = [
    [  0.0,   0.0,   0.0],   # binary A center
    [ 100.0,   50.0,   0.0],   # binary B center (nearby)
    [-300.0,   50.0,   0.0],   # binary C center (nearby)
]  # AU

# Small relative COM velocities so gravity pulls them together rather than flying apart.
v_coms = [
    [ 0.05,  0.00,  0.00],   # km/s
    [ 0.00, -0.05,  0.00],
    [ 0.00,  0.05,  0.00],
]

# Orbital orientation angles for each binary (radians). Use non-zero inclinations to allow
# large-angle interactions (Kozai-like orbits can help produce collisions).
directions = [0.0, 0.4, -0.4]

# Example run call
frames, max_mass, max_vel = run_6_body_simulation(
    age, masses, sep, ecc, directions, centers, v_coms, run_label="Run_for_ejection"
)

Starting simulation
Collision detected at 805.0 yr between 0 and 1
Fi collision done: {'M': quantity<106.838499325 MSun>, 'Rscale': quantity<0.452765091637 au>, 'N': 1000}
Collision 0 processed: remnant = 104.53 M☉, R = 14.16 R☉
Most massive star: Mass = 104.53 MSun, Velocity = 27.72 km/s


In [6]:
visualize_frames(frames, "Collision")

Tracking most massive star (index 0) with final mass 104.53 M☉
🎬 GIF saved as Gif-6body/encounter_evolution_Collision.gif


# MC integrations (wrong)

In [3]:
# run_MC_6body.py
"""
Minimal driver to run full Monte Carlo triple-triple encounters
using cross_section_6body_Hut.py
"""

import cross_section_6body_Hut  # your module
import os
import sys

# -----------------------------
# Parameters
# -----------------------------
v_factor = 1.0       # velocity in units of critical velocity
n_b = 5              # number of impact parameters
sim_per_b = 3        # simulations per impact parameter
output_dir = "MC_test_run"

os.makedirs(output_dir, exist_ok=True)

# -----------------------------
# Simulate command-line args
# -----------------------------
sys.argv = [
    "run_MC_6body.py",
    "--velocity", str(v_factor),
    "--n_b", str(n_b),
    "--sim_per_b", str(sim_per_b),
    "--output", output_dir
]

# -----------------------------
# Execute the main driver
# -----------------------------
if __name__ == "__main__":
    # Trick cross_section_6body_Hut to think it's run as __main__
    cross_section_6body_Hut.__name__ = "__main__"
    exec(open(cross_section_6body_Hut.__file__).read())




Results will be saved in MC_test_run
--- Impact parameter b=0.0 au ---
Starting simulation
Collision detected at 5.0 yr between 0 and 1
Fi collision done: {'M': quantity<132.735944114 MSun>, 'Rscale': quantity<0.353959785207 au>, 'N': 1000}
Collision 0 processed: remnant = 7.96 M☉, R = 3.26 R☉
Most massive star: Mass = 61.62 MSun, Velocity = 213.00 km/s
Starting simulation
Collision detected at 5.0 yr between 1 and 3
Fi collision done: {'M': quantity<98.5536174915 MSun>, 'Rscale': quantity<0.241709749552 au>, 'N': 1000}
Collision 0 processed: remnant = 51.39 M☉, R = 9.44 R☉
Most massive star: Mass = 66.46 MSun, Velocity = 120.62 km/s
Starting simulation


KeyboardInterrupt: 